In [166]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance



In [168]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [169]:
!ls

data  day2_visualisation.ipynb	day3_simple_model.ipynb  LICENSE  README.md


In [170]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list):continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat +SUFFIX_CAT] = factorized_values

In [172]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):

  X = df[feats].values
  Y = df['price_value'].values

  
  scores = cross_val_score(model, X, Y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [174]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015937)

##Random Forest

In [175]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

##XGBoost

In [176]:
xgb_params = {'max_depth':5, 'n_estimators':50, 'learnin_rate': 0.1, 'seed': 0}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

[14:05:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [178]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learnin_rate=0.1, seed= 0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp,feature_names=cat_feats)

[14:06:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [202]:
len(cat_feats)

151

In [0]:
  feats = [
  'param_napęd__cat',
  'param_rok-produkcji__cat',
  'param_stan__cat',
  'param_skrzynia-biegów__cat',
  'param_faktura-vat__cat',
  'param_moc__cat',
  'param_marka-pojazdu__cat',
  'feature_kamera-cofania__cat',
  'param_typ__cat',
  'param_pojemność-skokowa__cat',
  'seller_name__cat',
  'feature_wspomaganie-kierownicy__cat',
  'param_model-pojazdu__cat',
  'param_wersja__cat',
  'param_kod-silnika__cat',
  'feature_system-start-stop__cat',
  'feature_asystent-pasa-ruchu__cat',
  'feature_czujniki-parkowania-przednie__cat',
  'feature_łopatki-zmiany-biegów__cat',
  'feature_regulowane-zawieszenie__cat',]

In [213]:
len(feats)

20

In [228]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[15:03:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:03:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:03:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [230]:
  feats = [
  'param_napęd__cat',
  'param_rok-produkcji',
  'param_stan__cat',
  'param_skrzynia-biegów__cat',
  'param_faktura-vat__cat',
  'param_moc__cat',
  'param_marka-pojazdu__cat',
  'feature_kamera-cofania__cat',
  'param_typ__cat',
  'param_pojemność-skokowa__cat',
  'seller_name__cat',
  'feature_wspomaganie-kierownicy__cat',
  'param_model-pojazdu__cat',
  'param_wersja__cat',
  'param_kod-silnika__cat',
  'feature_system-start-stop__cat',
  'feature_asystent-pasa-ruchu__cat',
  'feature_czujniki-parkowania-przednie__cat',
  'feature_łopatki-zmiany-biegów__cat',
  'feature_regulowane-zawieszenie__cat',]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[15:04:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [232]:
df['param_moc'] = df['param_moc'].map(lambda x:  -1 if str(x) == 'None' else int(x.split(' ')[0]))

AttributeError: ignored

In [234]:
feats = [
  'param_napęd__cat',
  'param_rok-produkcji',
  'param_stan__cat',
  'param_skrzynia-biegów__cat',
  'param_faktura-vat__cat',
  'param_moc',
  'param_marka-pojazdu__cat',
  'feature_kamera-cofania__cat',
  'param_typ__cat',
  'param_pojemność-skokowa__cat',
  'seller_name__cat',
  'feature_wspomaganie-kierownicy__cat',
  'param_model-pojazdu__cat',
  'param_wersja__cat',
  'param_kod-silnika__cat',
  'feature_system-start-stop__cat',
  'feature_asystent-pasa-ruchu__cat',
  'feature_czujniki-parkowania-przednie__cat',
  'feature_łopatki-zmiany-biegów__cat',
  'feature_regulowane-zawieszenie__cat',]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[15:07:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:07:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:08:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [235]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:  -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [0]:
feats = [
  'param_napęd__cat',
  'param_rok-produkcji',
  'param_stan__cat',
  'param_skrzynia-biegów__cat',
  'param_faktura-vat__cat',
  'param_moc',
  'param_marka-pojazdu__cat',
  'feature_kamera-cofania__cat',
  'param_typ__cat',
  'param_pojemność-skokowa',
  'seller_name__cat',
  'feature_wspomaganie-kierownicy__cat',
  'param_model-pojazdu__cat',
  'param_wersja__cat',
  'param_kod-silnika__cat',
  'feature_system-start-stop__cat',
  'feature_asystent-pasa-ruchu__cat',
  'feature_czujniki-parkowania-przednie__cat',
  'feature_łopatki-zmiany-biegów__cat',
  'feature_regulowane-zawieszenie__cat',]

In [238]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[15:12:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:12:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:12:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)